In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import Xception
from os import getcwd

In [ ]:
# use xception transfer learning with input shape 300,300
pre_trained=Xception(
    include_top=False,
    input_shape=(300,300,3),
    weights='imagenet'
)

83689472/83683744 [==============================] - 1s 0us/step


In [ ]:
# no need to retrain the layers
for layer in pre_trained.layers:
  layer.trainable=False

In [ ]:
# combine the pre-trained layers with our self-built layers
inputs=Input(shape=(300,300,3))
x=pre_trained(inputs, training=False)
x=layers.GlobalAveragePooling2D()(x) # use global average pooling instead of flatten to make it faster
outputs=layers.Dense(1, activation='sigmoid')(x)

model=Model(inputs,outputs)
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=0.01),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# x=layers.Flatten()(pre_trained.output)
# x=layers.Dense(1024, activation='relu')(x)
# x=layers.Dropout(0.2)(x)
# x=layers.Dense(1, activation='sigmoid')(x)

# model=Model(pre_trained.input,x)
# model.compile(
#     optimizer=tf.keras.optimizers.Adam(lr=0.01),
#     loss='binary_crossentropy',
#     metrics=['accuracy']
# )

In [ ]:
TRAIN_DIR='/content/train'
VAL_DIR='/content/validation'
TEST_DIR='/content/test'

In [ ]:
# train_datagen=ImageDataGenerator(1./255)
# train_generator=train_datagen.flow_from_directory(
#     TRAIN_DIR,
#     target_size=(300,300),
#     batch_size=64,
#     class_mode='binary'
# )

Found 57671 images belonging to 2 classes.


In [ ]:
# do some augmentation techniques (meanwhile we just saved the non-augmented model as it gave better accuracy)
train_datagen=ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_generator=train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(300,300),
    batch_size=64,
    class_mode='binary'
)

Found 57671 images belonging to 2 classes.


In [ ]:
validation_datagen=ImageDataGenerator(1./255)
validation_generator=validation_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(300,300),
    batch_size=64,
    class_mode='binary'
)

Found 11860 images belonging to 2 classes.


In [ ]:
# import os
# from PIL import Image
# folder_path = '/content/test'
# lol=[]
# extensions = []
# for fldr in os.listdir(folder_path):
#     sub_folder_path = os.path.join(folder_path, fldr)
#     for filee in os.listdir(sub_folder_path):
#         file_path = os.path.join(sub_folder_path, filee)
#         print('** Path: {}  **'.format(file_path), end="\r", flush=True)
#         if file_path in lol:
#           continue
#         im = Image.open(file_path)
#         rgb_im = im.convert('RGB')
#         if filee.split('.')[1] not in extensions:
#             extensions.append(filee.split('.')[1])

In [ ]:
# train the model
history=model.fit_generator(
    train_generator,
    epochs=5,
    verbose=1,
    validation_data=validation_generator
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
902/902 [==============================] - ETA: 0s - loss: 1.4800 - accuracy: 0.9408

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


902/902 [==============================] - 1243s 1s/step - loss: 1.4800 - accuracy: 0.9408 - val_loss: 0.5095 - val_accuracy: 0.9745
Epoch 2/5
902/902 [==============================] - 1247s 1s/step - loss: 1.1983 - accuracy: 0.9438 - val_loss: 1.3225 - val_accuracy: 0.9737
Epoch 3/5
902/902 [==============================] - 1228s 1s/step - loss: 1.1257 - accuracy: 0.9481 - val_loss: 0.8521 - val_accuracy: 0.9589
Epoch 4/5
902/902 [==============================] - 1222s 1s/step - loss: 0.9661 - accuracy: 0.9476 - val_loss: 0.6648 - val_accuracy: 0.9715
Epoch 5/5
902/902 [==============================] - 1230s 1s/step - loss: 0.9596 - accuracy: 0.9490 - val_loss: 0.8974 - val_accuracy: 0.9390


In [ ]:
test_datagen=ImageDataGenerator(1./255)
test_generator=test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(300,300),
    batch_size=64,
    class_mode='binary'
)

Found 18220 images belonging to 2 classes.


In [ ]:
# evaluate the model with test data
loss,acc=model.evaluate(test_generator, verbose=2)
print(acc)

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


285/285 - 146s - loss: 2.3385 - accuracy: 0.8650
0.8649835586547852


In [ ]:
# save it into .h5 format
model.save('/content/drive/MyDrive/models/saved_model/xception_new_aug.h5')